In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast


from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"
AIRFLOW_WASSTRAAT_CONFIG_NEW = "./wasstraat_config/Wasstraat_Config_Harmonize_new.xlsx"

SUGGESTIE_XL = "./wasstraat_config/Wasstraat_Attribuut_Suggestie_v2.xlsx"

In [3]:
xl_config = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, None);
df_table = xl_config['Artefact']
df_table['Kolommen'] = df_table.apply(lambda x: ast.literal_eval(x['Kolommen']), axis=1)

df_table.head(5)

,Attribute,Kolommen,Unnamed: 2
0,putnr,"[PUT, PUTNO, put]",NaN
1,vlaknr,"[VLAK, VLAKNO]",NaN
2,spoornr,"[SPOOR, SPOORNO]",NaN
3,vondstnr,"[VONDSTNO, Vondst, vondstnr, VONDST]",NaN
4,doosnr,"[DOOSNO, 10a, DOOS_NO]",NaN


In [4]:
xl_sugg = pd.read_excel(SUGGESTIE_XL, None);
df_sugg = xl_sugg['Aardewerk']
df_sugg.head(5)

,Unnamed: 0,Object,Kolommen,omschrijvingen,count,projecten,Attribute,teller,percentage_gevuld,sugg_kolom,sugg_omschr,new_attr,new_attr_artefact
0,104,Aardewerk,VORM,['vorm'],4,"['DC179', 'DC008', 'DB34', 'DC24_STADSKANTOOR']",NaN,148.0,2.0,vorm,vorm,vorm,NaN
1,103,Aardewerk,VOORWERP,['type voorwerp'],1,['DB34'],NaN,NaN,NaN,voorwerp,type,NaN,typevoorwerp
2,129,Aardewerk,vondstnr,[],1,['DC24_STADSKANTOOR'],vondstnr,880.0,11.0,vondstnr,NaN,NaN,NaN
3,102,Aardewerk,VONDSTNO,['vondstnummer'],11,"['DC72', 'DC112', 'DC36', 'DC20', 'DB34', 'DC0...",vondstnr,7090.0,88.0,vondstno,vondstnr,NaN,NaN
4,128,Aardewerk,vondst,[],1,['DC24_STADSKANTOOR'],NaN,880.0,11.0,vondst,NaN,NaN,vondstnr


In [5]:
def combineDefs(df_config, df_sugg):
    df_config = df_config.append(df_sugg)
    df_config = df_config.groupby('Attribute').agg({'Kolommen':lambda x: list(x), 'Opmerkingen':lambda x: list(x)}).reset_index()
    df_config['Kolommen'] = df_config.apply(lambda x: list(set([item for sublist in x['Kolommen'] for item in sublist])), axis=1)
    df_config['Opmerkingen'] = df_config.apply(lambda x: ','.join([elem for elem in x['Opmerkingen'] if not pd.isnull(elem)]), axis=1)
    return df_config

    
kolommen = ['Attribute', 'Kolommen', 'Opmerkingen']
sheets_sugg = xl_sugg.keys()
sheets_config = xl_config.keys()
new_config = {}


df_artefact = pd.DataFrame()
for sheet in sheets_config:
    df_config = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, sheet)
    if sheet in sheets_sugg:
        df_sugg = pd.read_excel(SUGGESTIE_XL, sheet)
        df_sugg = df_sugg.groupby('new_attr_artefact').agg({'Kolommen':lambda x: list(set(x))})
        df_artefact = df_artefact.append(df_sugg.copy()) 
        
        # Now find suggestions for this one tab
        df_sugg = pd.read_excel(SUGGESTIE_XL, sheet)
        df_sugg = df_sugg.groupby('new_attr').agg({'Kolommen':lambda x: list(set(x))})
        df_sugg.reset_index(inplace=True)
        df_sugg['Opmerkingen'] = None
        df_sugg.columns = kolommen
        
        #Check Config
        if len(df_config) > 0:
            df_config['Kolommen'] = df_config.apply(lambda x: ast.literal_eval(x['Kolommen']), axis=1)
            if len(df_config.columns) < 3:
                df_config['Opmerkingen'] = None
            df_config.columns = kolommen
        else: 
            df_config = pd.DataFrame(columns = kolommen)

        df_config = combineDefs(df_config, df_sugg)
    
    new_config[sheet] = df_config
    
#df_sugg = pd.read_excel(SUGGESTIE_XL, 'Aardewerk')
#df_sugg = df_sugg.groupby('new_attr_artefact').agg({'Kolommen':lambda x: list(set(x))})
#df_sugg

df_config_artefact = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, 'Artefact')
df_config_artefact['Kolommen'] = df_config_artefact.apply(lambda x: ast.literal_eval(x['Kolommen']), axis=1)
df_config_artefact.columns = kolommen

df_artefact.reset_index(inplace=True)
df_artefact['Opmerkingen'] = None
df_artefact.columns = kolommen

df_config_artefact = combineDefs(df_config_artefact, df_artefact)
new_config['Artefact'] = df_config_artefact


new_config['Glas']

,Attribute,Kolommen,Opmerkingen
0,compleetheid,"[COMPLEET, 6d]",
1,decoratie,"[5c, decoratie]",
2,diameter_bodem,"[DB, DIA_BOD]",
3,diameter_rand,"[RANDDIA, DR]",
4,diversen,"[5d, DIVERSE]",
5,glassoort,"[GLSSOORT, glassoort, 5a]",
6,groep,[GROEP],
7,grootste,"[DIAGR, >D]",
8,hoogte,"[H, HOOGTE]",
9,kleur,"[KLEUR, kleur, 5b]",


In [9]:
with pd.ExcelWriter(AIRFLOW_WASSTRAAT_CONFIG_NEW) as writer:  
    for sheet in new_config.keys():
        new_config[sheet].to_excel(writer, sheet_name=sheet)


In [22]:
txt = '''artefactnr = Column(Integer)
    beschrijving = Column(Text)
    opmerkingen = Column(Text)
    typevoorwerp = Column(String(200))
    typecd = Column(String(200))
    functievoorwerp = Column(String(200))
    origine = Column(String(200))
    dateringvanaf = Column(Integer)
    dateringtot = Column(Integer)
    datering = Column(String(200))
    conserveren = Column(Integer)
    exposabel = Column(Integer)
    literatuur = Column(String(200))
    putnr = Column(Integer)
    subnr = Column(Integer) 
    restauratie = Column(Integer)'''

lst_aanw = [line.rpartition('=')[0].strip() for line in iter(txt.splitlines())] 
lst_aanw

['artefactnr',
 'beschrijving',
 'opmerkingen',
 'typevoorwerp',
 'typecd',
 'functievoorwerp',
 'origine',
 'dateringvanaf',
 'dateringtot',
 'datering',
 'conserveren',
 'exposabel',
 'literatuur',
 'putnr',
 'subnr',
 'restauratie']

In [23]:
col = 'typevoorwerp = Column(String(200))'

df_artf = new_config['Artefact']
for index, row in df_artf.iterrows():
    if not row['Attribute'] in lst_aanw:
        print(row['Attribute'] + ' = Column(String(200))')
    


ABR-codering = Column(String(200))
aantal = Column(String(200))
afmetingen = Column(String(200))
bibliografie = Column(String(200))
catalogus = Column(String(200))
compleetheid = Column(String(200))
conservering = Column(String(200))
dianr = Column(String(200))
diversen = Column(String(200))
doosnr = Column(String(200))
fotonr = Column(String(200))
gewicht = Column(String(200))
groep = Column(String(200))
mai = Column(String(200))
maten = Column(String(200))
materiaal = Column(String(200))
naam_voorwerp = Column(String(200))
plek = Column(String(200))
publicatiecode = Column(String(200))
soortvoorwerp = Column(String(200))
spoornr = Column(String(200))
tekeningnr = Column(String(200))
typenaam = Column(String(200))
versiering = Column(String(200))
vlaknr = Column(String(200))
vondstnr = Column(String(200))
vondstomstandigheden = Column(String(200))
weggegooid = Column(String(200))


In [19]:
for line in iter(txt.splitlines()):
    stripped_line = line.strip()
    print(stripped_line.rpartition('=')[0].strip())


artefactnr
beschrijving
opmerkingen
typevoorwerp
typecd
functievoorwerp
origine
dateringvanaf
dateringtot
datering
conserveren
exposabel
literatuur
putnr
subnr
restauratie



In [ ]:
ABR-codering = Column(String(200))
aantal = Column(Integer)
afmetingen = Column(String(200))
bibliografie = Column(String(200))
catalogus = Column(String(200))
compleetheid = Column(String(200))
conservering = Column(String(200))
diversen = Column(String(200))
gewicht = Column(String(200))
groep = Column(String(200))
mai = Column(String(200))
maten = Column(String(200))
materiaal = Column(String(200))
naam_voorwerp = Column(String(200))
plek = Column(String(200))
publicatiecode = Column(String(200))
soortvoorwerp = Column(String(200))
typenaam = Column(String(200))
versiering = Column(String(200))
vondstomstandigheden = Column(String(200))
weggegooid = Column(Integer)